<h2><b>CREDIT CARD FRAUD DETECTION

<ul>
  <li><strong>index:</strong> Unique Identifier for each row</li>
  <li><strong>trans_date_trans_time:</strong> Transaction DateTime</li>
  <li><strong>cc_num:</strong> Credit Card Number of Customer</li>
  <li><strong>merchant:</strong> Merchant Name</li>
  <li><strong>category:</strong> Category of Merchant</li>
  <li><strong>amt:</strong> Amount of Transaction</li>
  <li><strong>first:</strong> First Name of Credit Card Holder</li>
  <li><strong>last:</strong> Last Name of Credit Card Holder</li>
  <li><strong>gender:</strong> Gender of Credit Card Holder</li>
  <li><strong>street:</strong> Street Address of Credit Card Holder</li>
  <li><strong>city:</strong> City of Credit Card Holder</li>
  <li><strong>state:</strong> State of Credit Card Holder</li>
  <li><strong>zip:</strong> Zip of Credit Card Holder</li>
  <li><strong>lat:</strong> Latitude Location of Credit Card Holder</li>
  <li><strong>long:</strong> Longitude Location of Credit Card Holder</li>
  <li><strong>city_pop:</strong> Credit Card Holder's City Population</li>
  <li><strong>job:</strong> Job of Credit Card Holder</li>
  <li><strong>dob:</strong> Date of Birth of Credit Card Holder</li>
  <li><strong>trans_num:</strong> Transaction Number</li>
  <li><strong>unix_time:</strong> UNIX Time of transaction</li>
  <li><strong>merch_lat:</strong> Latitude Location of Merchant</li>
  <li><strong>merch_long:</strong> Longitude Location of Merchant</li>
  <li><strong>is_fraud:</strong> Fraud Flag &lt;--- Target Class</li>
</ul>


In [1]:
import pandas as pd 
import numpy as np

In [2]:
df=pd.read_csv('./archive/fraudTrain.csv',usecols=['cc_num','amt','zip','lat','long','city_pop','is_fraud'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 7 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   cc_num    1296675 non-null  int64  
 1   amt       1296675 non-null  float64
 2   zip       1296675 non-null  int64  
 3   lat       1296675 non-null  float64
 4   long      1296675 non-null  float64
 5   city_pop  1296675 non-null  int64  
 6   is_fraud  1296675 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 69.3 MB


In [4]:
df.isnull().sum()

cc_num      0
amt         0
zip         0
lat         0
long        0
city_pop    0
is_fraud    0
dtype: int64

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
X=df.drop('is_fraud',axis=1)
y=df['is_fraud']

In [7]:
df.corr()['is_fraud']

cc_num     -0.001078
amt         0.218494
zip        -0.002370
lat         0.002052
long        0.001864
city_pop    0.002196
is_fraud    1.000000
Name: is_fraud, dtype: float64

In [8]:
from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression

In [9]:
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import train_test_split 
X_train ,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

method= SMOTE()
X_resampled, y_resampled = method.fit_resample(X_train, y_train)

<b><h3>Transformers

In [10]:
trf1=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,6))
])
trf2=SelectKBest(score_func=chi2,k=5)
trf3=LogisticRegression()

<h3><b>Pipeline

In [11]:
pipe =Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3)
])
pipe.fit(X_resampled,y_resampled)

Pipeline(steps=[('trf1',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 6, None))])),
                ('trf2',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x00000264307CE5E0>)),
                ('trf3', LogisticRegression())])

In [12]:
y_pred=pipe.predict(X_test)


In [13]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
print('Classification report:\n', classification_report(y_test, y_pred))
conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print('Share of Non-Fraud in Test Data:', 1-round(y_test.sum()/len(y_test),4))

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98    231861
           1       0.10      0.75      0.18      1492

    accuracy                           0.96    233353
   macro avg       0.55      0.85      0.58    233353
weighted avg       0.99      0.96      0.97    233353

Confusion matrix:
 [[222151   9710]
 [   372   1120]]
Share of Non-Fraud in Test Data: 0.9936


In [14]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test,y_pred)

In [15]:
# from sklearn.metrics import precision_score

# precision = precision_score(y_test, y_pred)
# print(f"Precision: {precision}")

In [16]:
# from sklearn.metrics import f1_score

# f1 = f1_score(y_test, y_pred)
# print(f"F1 Score: {f1}")

In [17]:
testdata=pd.read_csv('./archive/fraudTest.csv',usecols=['cc_num','amt','zip','lat','long','city_pop','is_fraud'])

In [18]:
y=testdata['is_fraud']
x=testdata.drop('is_fraud',axis=1)

In [19]:
y_pred=pipe.predict(x)

In [20]:
accuracy_score(y,y_pred)

0.961761249840297